In [2]:
from pandas import DataFrame, Series
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [4]:
# read in the main training csv
train_main = pd.read_csv('home-credit-default-risk/application_train.csv', na_values=['','XNA','XAP'])

In [5]:
print(train_main.shape)
train_main[:5]

(307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [6]:
# Drop records that are too sparse (after previous discovery, these are usually not represented in test data
# Family Status: Unknown
train_main = train_main[train_main['NAME_FAMILY_STATUS'] != 'Unknown'] #2 records with unknown family status
train_main = train_main[train_main['NAME_INCOME_TYPE'] != 'Businessman'] #10 businessman

In [7]:
train_main.shape # 12 records dropped

(307499, 122)

In [8]:
# read in the ancillary csv's
credit_card = pd.read_csv('cleaned_tables/credit_card_summary.csv')
POS = pd.read_csv('cleaned_tables/df_POS_final2.csv')
install = pd.read_csv('cleaned_tables/df_pay_final2.csv')
bureau = pd.read_csv('cleaned_tables/Bureau_features.csv')
prev = pd.read_csv('cleaned_tables/prevapp_finally_train.csv')

### Get column names of additional tables

In [9]:
cred_col = credit_card.columns
credit_col = cred_col[1:]
len(credit_col) #8 extra

8

In [10]:
POS.drop(['DPD_df_POS','count_df_POS'], axis = 1, inplace = True)

In [11]:
POS_col = POS.columns[1:]
len(POS_col) #4 extra

4

In [12]:
install_col = install.columns[1:]
len(install_col) #12 extra

12

In [13]:
bureau_col = bureau.columns[1:]
len(bureau_col) #15 extra

15

In [14]:
prev_col = prev.columns[1:]
len(prev_col) # 10 extra

10

### merge tables

In [15]:
prev[:5]

,ID_CURR,AMT_APPLICATION,NUM_APPLICATIONS,AMT_APPROVED,PCT_APPROVED,MIN_DAYS_DECISION,PCT_APPL_ON_SAME_DAY,min_term,max_term,mean_term,DAYS_TERMINATION
0,100002,179055.00,1,179055.0,1.000000,-606,0.0,24.0,24.0,24.0,-17.0
1,100003,1306309.50,3,1452573.0,1.000000,-746,0.0,6.0,12.0,9.0,-527.0
2,100004,24282.00,1,20106.0,1.000000,-815,0.0,4.0,4.0,4.0,-714.0
3,100006,2449829.34,9,1718644.5,0.555556,-181,0.0,0.0,48.0,18.0,0.0
4,100007,903181.50,6,999832.5,1.000000,-374,0.0,18.0,48.0,30.0,0.0


In [16]:
df = train_main.merge(credit_card, how = 'left', on = 'SK_ID_CURR')
df = df.merge(POS, how = 'left', left_on ='SK_ID_CURR', right_on = 'SK_ID_CURR_df_POS').drop('SK_ID_CURR_df_POS', axis = 1)
df = df.merge(install, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR_pay').drop('SK_ID_CURR_pay', axis = 1)
df = df.merge(bureau, how = 'left', on = 'SK_ID_CURR')
df = df.merge(prev, how = 'left', left_on = 'SK_ID_CURR', right_on = 'ID_CURR').drop('ID_CURR',axis = 1)

In [17]:
df[:5]

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AVG_CC_BALANCE,MAX_CC_BALANCE,MIN_CC_BALANCE,MED_CC_BALANCE,MAX_CC_DPD,MIN_CC_DPD,AVG_CC_DPD,ZERO_CC_DPD,max_df_POS,min_df_POS,mean_df_POS,percentage_no_DPD_df_POS,NUM_INSTALMENT_NUMBER_pay,DPD_max_pay,DPD_min_pay,DPD_avg_pay,DPD_Count_pay,AMT_INSTALMENT_pay,AMT_PAYMENT_pay,PERCENT_NOT_PAID_pay,OWED_pay,PERCENT_UNDER_PAID_pay,PERCENT_OWED_pay,no_DPD_Percentage_pay,Percent_BU_CREDIT_ACTIVE,COUNT_BU_CREDIT_ACTIVE,AVG_BU_DAYS_CREDIT,MAX_BU_CREDIT_DAY_OVERDUE,MIN_BU_CREDIT_DAY_OVERDUE,AVG_BU_CREDIT_DAY_OVERDUE,percent_BU_CREDIT_DAY_OVERDUE,MAX_BU_DAYS_CREDIT_ENDDATE,MAX_BU_DAYS_ENDDATE,AVG_BU_CNT_CREDIT_PROLONG,MAX_BU_CNT_CREDIT_PROLONG,SUM_BU_AMT_CREDIT_SUM,COUNT_BU_CREDIT_TYPE,MAX_BU_PERCENT_SUM_OVERDUE,AVG_BU_PERCENT_SUM_OVERDUE,AMT_APPLICATION,NUM_APPLICATIONS,AMT_APPROVED,PCT_APPROVED,MIN_DAYS_DECISION,PCT_APPL_ON_SAME_DAY,min_term,max_term,mean_term,DAYS_TERMINATION
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,19.0,-12.0,-31.0,-20.421053,0.0,219625.695,219625.695,0.0,0.000,0.000000,0.000000,1.000000,0.00,4.0,-645.0,0.0,0.0,0.0,0.0,85.0,-36.0,0.0,0.0,315

### Fill nas for the summarized columns
if they did not appear in other tables, they did not have previous applications, credit cards, etc.

In [18]:
addl_col = cred_col.append([POS_col, install_col, bureau_col, prev_col]) # append other tables' columns as well
df[addl_col] = df[addl_col].fillna(0)
df[:5]

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AVG_CC_BALANCE,MAX_CC_BALANCE,MIN_CC_BALANCE,MED_CC_BALANCE,MAX_CC_DPD,MIN_CC_DPD,AVG_CC_DPD,ZERO_CC_DPD,max_df_POS,min_df_POS,mean_df_POS,percentage_no_DPD_df_POS,NUM_INSTALMENT_NUMBER_pay,DPD_max_pay,DPD_min_pay,DPD_avg_pay,DPD_Count_pay,AMT_INSTALMENT_pay,AMT_PAYMENT_pay,PERCENT_NOT_PAID_pay,OWED_pay,PERCENT_UNDER_PAID_pay,PERCENT_OWED_pay,no_DPD_Percentage_pay,Percent_BU_CREDIT_ACTIVE,COUNT_BU_CREDIT_ACTIVE,AVG_BU_DAYS_CREDIT,MAX_BU_CREDIT_DAY_OVERDUE,MIN_BU_CREDIT_DAY_OVERDUE,AVG_BU_CREDIT_DAY_OVERDUE,percent_BU_CREDIT_DAY_OVERDUE,MAX_BU_DAYS_CREDIT_ENDDATE,MAX_BU_DAYS_ENDDATE,AVG_BU_CNT_CREDIT_PROLONG,MAX_BU_CNT_CREDIT_PROLONG,SUM_BU_AMT_CREDIT_SUM,COUNT_BU_CREDIT_TYPE,MAX_BU_PERCENT_SUM_OVERDUE,AVG_BU_PERCENT_SUM_OVERDUE,AMT_APPLICATION,NUM_APPLICATIONS,AMT_APPROVED,PCT_APPROVED,MIN_DAYS_DECISION,PCT_APPL_ON_SAME_DAY,min_term,max_term,mean_term,DAYS_TERMINATION
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,19.0,-12.0,-31.0,-20.421053,0.0,219625.695,219625.695,0.0,0.000,0.000000,0.000000,1.000000,0.00,4.0,-645.0,0.0,0.0,0.0,0.0,85.0,-36.0,0.0,0.0,315

### Required preprocessing steps for the whole table
1. Split train / test
2. Fill NAs - following the decision made when exploring the main table
3. Set up categorical variables
4. Rebalance / SMOTE (part of modeling)

### 1. Split train/test

In [19]:
X = df[df.columns[2:]]
print(X.shape)
X[:2]

(307499, 169)


,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AVG_CC_BALANCE,MAX_CC_BALANCE,MIN_CC_BALANCE,MED_CC_BALANCE,MAX_CC_DPD,MIN_CC_DPD,AVG_CC_DPD,ZERO_CC_DPD,max_df_POS,min_df_POS,mean_df_POS,percentage_no_DPD_df_POS,NUM_INSTALMENT_NUMBER_pay,DPD_max_pay,DPD_min_pay,DPD_avg_pay,DPD_Count_pay,AMT_INSTALMENT_pay,AMT_PAYMENT_pay,PERCENT_NOT_PAID_pay,OWED_pay,PERCENT_UNDER_PAID_pay,PERCENT_OWED_pay,no_DPD_Percentage_pay,Percent_BU_CREDIT_ACTIVE,COUNT_BU_CREDIT_ACTIVE,AVG_BU_DAYS_CREDIT,MAX_BU_CREDIT_DAY_OVERDUE,MIN_BU_CREDIT_DAY_OVERDUE,AVG_BU_CREDIT_DAY_OVERDUE,percent_BU_CREDIT_DAY_OVERDUE,MAX_BU_DAYS_CREDIT_ENDDATE,MAX_BU_DAYS_ENDDATE,AVG_BU_CNT_CREDIT_PROLONG,MAX_BU_CNT_CREDIT_PROLONG,SUM_BU_AMT_CREDIT_SUM,COUNT_BU_CREDIT_TYPE,MAX_BU_PERCENT_SUM_OVERDUE,AVG_BU_PERCENT_SUM_OVERDUE,AMT_APPLICATION,NUM_APPLICATIONS,AMT_APPROVED,PCT_APPROVED,MIN_DAYS_DECISION,PCT_APPL_ON_SAME_DAY,min_term,max_term,mean_term,DAYS_TERMINATION
0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,19.0,-12.0,-31.0,-20.421053,0.0,219625.695,219625.695,0.0,0.0,0.0,0.0,1.0,0.00,4.0,-645.0,0.0,0.0,0.0,0.0,85.0,-36.0,0.0,0.0,315567.0,2.0,0.0,0.0,179055.0,1.0,179055.0,1.0,

In [20]:
y = df['TARGET']
print(y.shape)

(307499,)


In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [22]:
print(len(X_train),len(X_test),len(y_train),len(y_test))
print(len(y_train[y_train == 0])/len(y_train),len(y_test[y_test == 0])/len(y_test))

245999 61500 245999 61500
0.9190728417595193 0.9200487804878049


### 2. Fill in missing values

In [23]:
# Drop highly correlated "current housing" attributes
X_train_nona = X_train.drop(['APARTMENTS_AVG',
'APARTMENTS_MODE',
'BASEMENTAREA_AVG',
'BASEMENTAREA_MODE',
'COMMONAREA_AVG',
'COMMONAREA_MODE',
'ELEVATORS_AVG',
'ELEVATORS_MODE',
'ENTRANCES_AVG',
'ENTRANCES_MODE',
'FLOORSMAX_AVG',
'FLOORSMAX_MODE',
'FLOORSMIN_AVG',
'FLOORSMIN_MODE',
'LANDAREA_AVG',
'LANDAREA_MODE',
'LIVINGAPARTMENTS_AVG',
'LIVINGAPARTMENTS_MODE',
'LIVINGAREA_AVG',
'LIVINGAREA_MODE',
'NONLIVINGAPARTMENTS_AVG',
'NONLIVINGAPARTMENTS_MODE',
'NONLIVINGAREA_AVG',
'NONLIVINGAREA_MODE',
'YEARS_BEGINEXPLUATATION_AVG',
'YEARS_BEGINEXPLUATATION_MEDI',
'YEARS_BEGINEXPLUATATION_MODE',
'YEARS_BUILD_AVG',
'YEARS_BUILD_MODE',
'DEF_60_CNT_SOCIAL_CIRCLE',
'OBS_60_CNT_SOCIAL_CIRCLE'], axis = 1)

X_test_nona = X_test.drop(['APARTMENTS_AVG',
'APARTMENTS_MODE',
'BASEMENTAREA_AVG',
'BASEMENTAREA_MODE',
'COMMONAREA_AVG',
'COMMONAREA_MODE',
'ELEVATORS_AVG',
'ELEVATORS_MODE',
'ENTRANCES_AVG',
'ENTRANCES_MODE',
'FLOORSMAX_AVG',
'FLOORSMAX_MODE',
'FLOORSMIN_AVG',
'FLOORSMIN_MODE',
'LANDAREA_AVG',
'LANDAREA_MODE',
'LIVINGAPARTMENTS_AVG',
'LIVINGAPARTMENTS_MODE',
'LIVINGAREA_AVG',
'LIVINGAREA_MODE',
'NONLIVINGAPARTMENTS_AVG',
'NONLIVINGAPARTMENTS_MODE',
'NONLIVINGAREA_AVG',
'NONLIVINGAREA_MODE',
'YEARS_BEGINEXPLUATATION_AVG',
'YEARS_BEGINEXPLUATATION_MEDI',
'YEARS_BEGINEXPLUATATION_MODE',
'YEARS_BUILD_AVG',
'YEARS_BUILD_MODE',
'DEF_60_CNT_SOCIAL_CIRCLE',
'OBS_60_CNT_SOCIAL_CIRCLE'], axis = 1)

In [24]:
print(X_train_nona.shape, X_test_nona.shape) # 31 columns dropped; 169 --> 138

(245999, 138) (61500, 138)


In [25]:
# Filling with Medians
columns_with_median = ['COMMONAREA_MEDI',
'LIVINGAPARTMENTS_MEDI',
'FLOORSMIN_MEDI',
'LANDAREA_MEDI',
'BASEMENTAREA_MEDI',
'ELEVATORS_MEDI',
'APARTMENTS_MEDI',
'ENTRANCES_MEDI',
'LIVINGAREA_MEDI',
'FLOORSMAX_MEDI',
'AMT_GOODS_PRICE',
'AMT_ANNUITY']

for col in columns_with_median:
    X_train_nona[col].fillna(X_train_nona[col].median(), inplace = True)
    X_test_nona[col].fillna(X_train_nona[col].median(), inplace = True)

In [26]:
# Filling with Means
columns_with_means = ['YEARS_BUILD_MEDI',
'EXT_SOURCE_1',
'TOTALAREA_MODE',
'EXT_SOURCE_3',
'EXT_SOURCE_2']

for col in columns_with_means:
    X_train_nona[col].fillna(X_train_nona[col].mean(), inplace = True)
    X_test_nona[col].fillna(X_train_nona[col].mean(), inplace = True)

In [27]:
# Fill with Mode
columns_with_modes = ['FONDKAPREMONT_MODE', 
'HOUSETYPE_MODE',
'EMERGENCYSTATE_MODE']

for col in columns_with_modes:
    X_train_nona[col].fillna(X_train_nona[col].mode()[0], inplace = True)
    X_test_nona[col].fillna(X_train_nona[col].mode()[0], inplace = True)

In [28]:
# Fill with 0's

columns_with_0s = ['NONLIVINGAPARTMENTS_MEDI',
'OWN_CAR_AGE',
'NONLIVINGAREA_MEDI',
'AMT_REQ_CREDIT_BUREAU_YEAR',
'AMT_REQ_CREDIT_BUREAU_DAY',
'AMT_REQ_CREDIT_BUREAU_QRT',
'AMT_REQ_CREDIT_BUREAU_WEEK',
'AMT_REQ_CREDIT_BUREAU_HOUR',
'AMT_REQ_CREDIT_BUREAU_MON',
'OBS_30_CNT_SOCIAL_CIRCLE',
'DEF_30_CNT_SOCIAL_CIRCLE',
'CNT_FAM_MEMBERS',
'DAYS_LAST_PHONE_CHANGE']

for col in columns_with_0s:
    X_train_nona[col].fillna(0, inplace = True)
    X_test_nona[col].fillna(0, inplace = True)

In [29]:
# Other special cases
X_train_nona['WALLSMATERIAL_MODE'].fillna("Others",inplace = True)
X_train_nona['OCCUPATION_TYPE'].fillna("Unemployed",inplace = True)
X_train_nona['ORGANIZATION_TYPE'].fillna("Retired",inplace = True)
X_train_nona['NAME_TYPE_SUITE'].fillna("Unaccompanied",inplace = True)
X_train_nona['CODE_GENDER'].fillna("F",inplace = True)

X_test_nona['WALLSMATERIAL_MODE'].fillna("Others",inplace = True)
X_test_nona['OCCUPATION_TYPE'].fillna("Unemployed",inplace = True)
X_test_nona['ORGANIZATION_TYPE'].fillna("Retired",inplace = True)
X_test_nona['NAME_TYPE_SUITE'].fillna("Unaccompanied",inplace = True)
X_test_nona['CODE_GENDER'].fillna("F",inplace = True)

In [30]:
# verify that there's no more missing values in the training data
(X_train_nona.isnull().sum() / len(X_train_nona)*100).sort_values(ascending = False)

DAYS_TERMINATION                 0.0
BASEMENTAREA_MEDI                0.0
FLOORSMIN_MEDI                   0.0
FLOORSMAX_MEDI                   0.0
ENTRANCES_MEDI                   0.0
ELEVATORS_MEDI                   0.0
COMMONAREA_MEDI                  0.0
YEARS_BUILD_MEDI                 0.0
APARTMENTS_MEDI                  0.0
LIVE_REGION_NOT_WORK_REGION      0.0
EXT_SOURCE_3                     0.0
EXT_SOURCE_2                     0.0
EXT_SOURCE_1                     0.0
ORGANIZATION_TYPE                0.0
LIVE_CITY_NOT_WORK_CITY          0.0
REG_CITY_NOT_WORK_CITY           0.0
LANDAREA_MEDI                    0.0
LIVINGAPARTMENTS_MEDI            0.0
LIVINGAREA_MEDI                  0.0
NONLIVINGAPARTMENTS_MEDI         0.0
NONLIVINGAREA_MEDI               0.0
FONDKAPREMONT_MODE               0.0
HOUSETYPE_MODE                   0.0
TOTALAREA_MODE                   0.0
WALLSMATERIAL_MODE               0.0
EMERGENCYSTATE_MODE              0.0
OBS_30_CNT_SOCIAL_CIRCLE         0.0
D

In [31]:
(X_test_nona.isnull().sum() / len(X_test_nona)*100).sort_values(ascending = False)

DAYS_TERMINATION                 0.0
BASEMENTAREA_MEDI                0.0
FLOORSMIN_MEDI                   0.0
FLOORSMAX_MEDI                   0.0
ENTRANCES_MEDI                   0.0
ELEVATORS_MEDI                   0.0
COMMONAREA_MEDI                  0.0
YEARS_BUILD_MEDI                 0.0
APARTMENTS_MEDI                  0.0
LIVE_REGION_NOT_WORK_REGION      0.0
EXT_SOURCE_3                     0.0
EXT_SOURCE_2                     0.0
EXT_SOURCE_1                     0.0
ORGANIZATION_TYPE                0.0
LIVE_CITY_NOT_WORK_CITY          0.0
REG_CITY_NOT_WORK_CITY           0.0
LANDAREA_MEDI                    0.0
LIVINGAPARTMENTS_MEDI            0.0
LIVINGAREA_MEDI                  0.0
NONLIVINGAPARTMENTS_MEDI         0.0
NONLIVINGAREA_MEDI               0.0
FONDKAPREMONT_MODE               0.0
HOUSETYPE_MODE                   0.0
TOTALAREA_MODE                   0.0
WALLSMATERIAL_MODE               0.0
EMERGENCYSTATE_MODE              0.0
OBS_30_CNT_SOCIAL_CIRCLE         0.0
D

Would need to do the same for testing to check model accuracy

### 3. Identify and codify categorical columns - X_train and X_test

### Identify categorical columns

In [32]:
# Categorical columns with strings
string_categorical = X_train_nona.columns[X_train_nona.dtypes == 'object']
s1 = df[string_categorical].nunique()

In [33]:
# string binary
binary_cat = s1[s1 ==2].index 
binary_cat

Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'EMERGENCYSTATE_MODE'],
      dtype='object')

In [34]:
# Categorical columns have numerical values
s2 = X_train_nona.loc[:,df.dtypes != 'object'].nunique()

In [35]:
# numerical binary
binary_cat = binary_cat.append(s2[s2==2].index[1:]) #numerical binary; exclude target

In [36]:
#all binary
binary_cat

Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'EMERGENCYSTATE_MODE', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE',
       'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL',
       'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
       'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY',
       'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'FLAG_DOCUMENT_2',
       'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5',
       'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8',
       'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11',
       'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14',
       'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17',
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21'],
      dtype='object')

In [37]:
# all non-binary categorical columns
non_bi_cat = s1[s1>2].index # string
non_bi_cat = non_bi_cat.append(s2[(s2>2) & (s2<5)].index) # numerical
non_bi_cat # all

Index(['NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
       'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
       'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'REGION_RATING_CLIENT',
       'REGION_RATING_CLIENT_W_CITY'],
      dtype='object')

In [38]:
print(len(binary_cat), len(non_bi_cat), len(X_train_nona.columns))

36 13 138


### Codify categorical columns
- Binary: label encoder
- Non-binary: get_dummies

In [39]:
# binary
for column in binary_cat:
    X_train_nona[column] = X_train_nona[column].astype('category').cat.codes
    X_test_nona[column] = X_test_nona[column].astype('category').cat.codes

In [40]:
# verifying everyone has two categories
X_train_nona[binary_cat].nunique()

NAME_CONTRACT_TYPE             2
CODE_GENDER                    2
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
EMERGENCYSTATE_MODE            2
FLAG_EMP_PHONE                 2
FLAG_WORK_PHONE                2
FLAG_CONT_MOBILE               2
FLAG_PHONE                     2
FLAG_EMAIL                     2
REG_REGION_NOT_LIVE_REGION     2
REG_REGION_NOT_WORK_REGION     2
LIVE_REGION_NOT_WORK_REGION    2
REG_CITY_NOT_LIVE_CITY         2
REG_CITY_NOT_WORK_CITY         2
LIVE_CITY_NOT_WORK_CITY        2
FLAG_DOCUMENT_2                2
FLAG_DOCUMENT_3                2
FLAG_DOCUMENT_4                2
FLAG_DOCUMENT_5                2
FLAG_DOCUMENT_6                2
FLAG_DOCUMENT_7                2
FLAG_DOCUMENT_8                2
FLAG_DOCUMENT_9                2
FLAG_DOCUMENT_10               2
FLAG_DOCUMENT_11               2
FLAG_DOCUMENT_12               2
FLAG_DOCUMENT_13               2
FLAG_DOCUMENT_14               2
FLAG_DOCUMENT_15               2
FLAG_DOCUM

In [41]:
# non-binary
X_train_nona = pd.get_dummies(X_train_nona,
                            prefix=non_bi_cat,
                            columns=non_bi_cat)
X_train_nona.shape #up from 138

(245999, 259)

In [42]:
X_test_nona = pd.get_dummies(X_test_nona,
                            prefix=non_bi_cat,
                            columns=non_bi_cat)
X_test_nona.shape #same with training

(61500, 259)

In [43]:
# X_train_nona.columns.intersection(X_test_nona.columns)
X_train_nona.columns.symmetric_difference(X_test_nona.columns) #identical sets of training predictors

Index([], dtype='object')

#### Check the processed, merged dataset

In [44]:
X_train_nona.describe()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AVG_CC_BALANCE,MAX_CC_BALANCE,MIN_CC_BALANCE,MED_CC_BALANCE,MAX_CC_DPD,MIN_CC_DPD,AVG_CC_DPD,ZERO_CC_DPD,max_df_POS,min_df_POS,mean_df_POS,percentage_no_DPD_df_POS,NUM_INSTALMENT_NUMBER_pay,DPD_max_pay,DPD_min_pay,DPD_avg_pay,DPD_Count_pay,AMT_INSTALMENT_pay,AMT_PAYMENT_pay,PERCENT_NOT_PAID_pay,OWED_pay,PERCENT_UNDER_PAID_pay,PERCENT_OWED_pay,no_DPD_Percentage_pay,Percent_BU_CREDIT_ACTIVE,COUNT_BU_CREDIT_ACTIVE,AVG_BU_DAYS_CREDIT,MAX_BU_CREDIT_DAY_OVERDUE,MIN_BU_CREDIT_DAY_OVERDUE,AVG_BU_CREDIT_DAY_OVERDUE,percent_BU_CREDIT_DAY_OVERDUE,MAX_BU_DAYS_CREDIT_ENDDATE,MAX_BU_DAYS_ENDDATE,AVG_BU_CNT_CREDIT_PROLONG,MAX_BU_CNT_CREDIT_PROLONG,SUM_BU_AMT_CREDIT_SUM,COUNT_BU_CREDIT_TYPE,MAX_BU_PERCENT_SUM_OVERDUE,AVG_BU_PERCENT_SUM_OVERDUE,AMT_APPLICATION,NUM_APPLICATIONS,AMT_APPROVED,PCT_APPROVED,MIN_DAYS_DECISION,PCT_APPL_ON_SAME_DAY,min_term,max_term,mean_term,DAYS_TERMINATION,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Unemployed,OCCUPATION_TYPE_Waiters/barmen staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_STAR

In [45]:
to observe external sources 2's and 3's correlation with biased attributes
X_train_nona.corr().to_csv("EDA-helpers/processed_correlation.csv")

# Modeling
- Training data: X_train_nona
- Training target: y_train
- Test data: X_test_nona
- Test target: y_test

## Decision Tree
### DT doesn't actually have ROC AUC score since there are no probabilities... the below are just for test

In [46]:
# set up classifier
from sklearn import tree
model = tree.DecisionTreeClassifier(criterion = 'entropy')

In [47]:
# Fit the data
result = model.fit(X_train_nona, y_train)

In [48]:
from sklearn import metrics
from sklearn.metrics import roc_auc_score, f1_score

prediction_train = model.predict(X_train_nona)
prediction_test = model.predict(X_test_nona)
y_score_test = model.predict_proba(X_test_nona)[:,1]
print("training accuracy:", metrics.accuracy_score(y_train, prediction_train),"\n"
      "test accuracy:", metrics.accuracy_score(y_test, prediction_test), "\n"
      "test ROC AUC score:", roc_auc_score(y_test,y_score_test), "\n"
     "test F1 score:", f1_score(y_test, prediction_test))

training accuracy: 1.0 
test accuracy: 0.8609918699186991 
test ROC AUC score: 0.5473863914157777 
test F1 score: 0.16684533671182147


In [49]:
pd.crosstab(y_test, prediction_test)

col_0,0,1
TARGET,,
0,52095,4488
1,4061,856


## Decision Tree - with Balanced Weight

In [50]:
# set up classifier
from sklearn import tree
model = tree.DecisionTreeClassifier(criterion = 'entropy', class_weight = 'balanced')

In [51]:
# Fit the data
result = model.fit(X_train_nona, y_train)

In [52]:
from sklearn import metrics
from sklearn.metrics import roc_auc_score, f1_score

prediction_train = model.predict(X_train_nona)
prediction_test = model.predict(X_test_nona)
y_score_test = model.predict_proba(X_test_nona)[:,1]
print("training accuracy:", metrics.accuracy_score(y_train, prediction_train),"\n"
      "test accuracy:", metrics.accuracy_score(y_test, prediction_test), "\n"
      "test ROC AUC score:", roc_auc_score(y_test,y_score_test), "\n"
     "test F1 score:", f1_score(y_test, prediction_test))

training accuracy: 1.0 
test accuracy: 0.8584390243902439 
test ROC AUC score: 0.5419135853568042 
test F1 score: 0.15721200387221682


In [53]:
pd.crosstab(y_test, prediction_test)

col_0,0,1
TARGET,,
0,51982,4601
1,4105,812


## Worse than the unbalanced tree...

# Logistic Regression

## 1. Logistic Regression - Regular

In [54]:
from sklearn.linear_model import LogisticRegression
lr1 = LogisticRegression()
lr1.fit(X_train_nona, y_train)

C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [55]:
prediction_train = lr1.predict(X_train_nona)
prediction_test = lr1.predict(X_test_nona)
y_score_test = lr1.predict_proba(X_test_nona)[:,1]
print("training accuracy:", metrics.accuracy_score(y_train, prediction_train),"\n"
      "test accuracy:", metrics.accuracy_score(y_test, prediction_test), "\n"
      "test ROC AUC score:", roc_auc_score(y_test,y_score_test), "\n"
     "test F1 score:", f1_score(y_test, prediction_test))

training accuracy: 0.919024061073419 
test accuracy: 0.9199349593495935 
test ROC AUC score: 0.6374826556804283 
test F1 score: 0.0


In [56]:
# find the best threshold
from sklearn.metrics import roc_curve
def Find_Optimal_Cutoff(target, predicted):
    """ Find the optimal probability cutoff point for a classification model related to event rate
    Parameters
    ----------
    target : Matrix with dependent or target data, where rows are observations

    predicted : Matrix with predicted data, where rows are observations

    Returns
    -------     
    list type, with optimal cutoff value

    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc_t = roc.ix[(roc.tf-0).abs().argsort()[:1]]

    return list(roc_t['threshold']) 

In [58]:
best_threshold = Find_Optimal_Cutoff(y_test, y_score_test)
best_f1_score = f1_score(y_test, (y_score_test > best_threshold))
print("The best F1 score", round(best_f1_score,8), "is achieved with the threshold of", round(best_threshold[0], 8))

The best F1 score 0.1948988 is achieved with the threshold of 0.08213694


C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated


In [121]:
from sklearn.metrics import confusion_matrix
# for precision/recall analysis
# precision = TP / (TP+FP)
# recall = TP / (TP+FN)
confusion_matrix(y_test, (y_score_test > best_threshold))

array([[34076, 22507],
       [ 1956,  2961]], dtype=int64)

## 2. Logistic Regression with Balanced Weight

In [59]:
lr2 = LogisticRegression(class_weight='balanced')
lr2.fit(X_train_nona, y_train)

C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [60]:
prediction_train_2 = lr2.predict(X_train_nona)
prediction_test_2 = lr2.predict(X_test_nona)
y_score_test_2 = lr2.predict_proba(X_test_nona)[:,1]
print("training accuracy:", metrics.accuracy_score(y_train, prediction_train_2),"\n"
      "test accuracy:", metrics.accuracy_score(y_test, prediction_test_2), "\n"
      "test ROC AUC score:", roc_auc_score(y_test,y_score_test_2), "\n"
     "test F1 score:", f1_score(y_test, prediction_test_2))

training accuracy: 0.6342586758482758 
test accuracy: 0.6331382113821138 
test ROC AUC score: 0.667870220227647 
test F1 score: 0.21144974136725847


In [61]:
best_threshold_2 = Find_Optimal_Cutoff(y_test, y_score_test_2)
best_f1_score_2 = f1_score(y_test, (y_score_test_2 > best_threshold_2))
print("The best F1 score", round(best_f1_score_2,8), "is achieved with the threshold of", round(best_threshold_2[0], 8))

# the function is an approximate; so the threshold 0.5 is already good for a balanced dataset

The best F1 score 0.20992888 is achieved with the threshold of 0.49688649


C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated


In [122]:
# for precision/recall analysis
# precision = TP / (TP+FP)
# recall = TP / (TP+FN)
confusion_matrix(y_test, (y_score_test_2 > best_threshold_2))

array([[35322, 21261],
       [ 1847,  3070]], dtype=int64)

## 3. Logistic Regression with SMOTENC

In [62]:
# prep SMOTENC dataset
X_train_S = pd.read_csv('cleaned_tables/processed_X_train_SMOTENC.csv', index_col = 0)
y_train_S = pd.read_csv('cleaned_tables/processed_y_train.csv', index_col = 0, header = None)

In [63]:
X_train_nona[:5]

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AVG_CC_BALANCE,MAX_CC_BALANCE,MIN_CC_BALANCE,MED_CC_BALANCE,MAX_CC_DPD,MIN_CC_DPD,AVG_CC_DPD,ZERO_CC_DPD,max_df_POS,min_df_POS,mean_df_POS,percentage_no_DPD_df_POS,NUM_INSTALMENT_NUMBER_pay,DPD_max_pay,DPD_min_pay,DPD_avg_pay,DPD_Count_pay,AMT_INSTALMENT_pay,AMT_PAYMENT_pay,PERCENT_NOT_PAID_pay,OWED_pay,PERCENT_UNDER_PAID_pay,PERCENT_OWED_pay,no_DPD_Percentage_pay,Percent_BU_CREDIT_ACTIVE,COUNT_BU_CREDIT_ACTIVE,AVG_BU_DAYS_CREDIT,MAX_BU_CREDIT_DAY_OVERDUE,MIN_BU_CREDIT_DAY_OVERDUE,AVG_BU_CREDIT_DAY_OVERDUE,percent_BU_CREDIT_DAY_OVERDUE,MAX_BU_DAYS_CREDIT_ENDDATE,MAX_BU_DAYS_ENDDATE,AVG_BU_CNT_CREDIT_PROLONG,MAX_BU_CNT_CREDIT_PROLONG,SUM_BU_AMT_CREDIT_SUM,COUNT_BU_CREDIT_TYPE,MAX_BU_PERCENT_SUM_OVERDUE,AVG_BU_PERCENT_SUM_OVERDUE,AMT_APPLICATION,NUM_APPLICATIONS,AMT_APPROVED,PCT_APPROVED,MIN_DAYS_DECISION,PCT_APPL_ON_SAME_DAY,min_term,max_term,mean_term,DAYS_TERMINATION,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Unemployed,OCCUPATION_TYPE_Waiters/barmen staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_STAR

In [64]:
print(X_train_S.shape, X_train_nona.shape, y_train_S.shape, y_train.shape)

(452182, 258) (245999, 259) (452182, 1) (245999,)


In [65]:
for column in X_test_nona.columns.values:
    if column not in X_train_S.columns.values:
        print(column)

AVG_CC_DPD


In [66]:
X_test_S = X_test_nona.loc[:, X_test_nona.columns != "AVG_CC_DPD"]
X_test_S.shape

(61500, 258)

In [67]:
# Train model
lr3 = LogisticRegression()
lr3.fit(X_train_S, y_train_S)

C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [68]:
prediction_train_3 = lr3.predict(X_train_S)
prediction_test_3 = lr3.predict(X_test_S)
y_score_test_3 = lr3.predict_proba(X_test_S)[:,1]
print("training accuracy:", metrics.accuracy_score(y_train_S, prediction_train_3),"\n"
      "test accuracy:", metrics.accuracy_score(y_test, prediction_test_3), "\n"
      "test ROC AUC score:", roc_auc_score(y_test,y_score_test_3), "\n"
     "test F1 score:", f1_score(y_test, prediction_test_3))

training accuracy: 0.5897249337656076 
test accuracy: 0.603739837398374 
test ROC AUC score: 0.6201494730343543 
test F1 score: 0.18728740078703396


In [69]:
best_threshold_3 = Find_Optimal_Cutoff(y_test, y_score_test_3)
best_f1_score_3 = f1_score(y_test, (y_score_test_3 > best_threshold_3))
print("The best F1 score", round(best_f1_score_3,8), "is achieved with the threshold of", round(best_threshold_3[0], 8))

# the function is an approximate; so the threshold 0.5 is already good for a SMOTED dataset

The best F1 score 0.18507827 is achieved with the threshold of 0.49642404


C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated


In [123]:
# for precision/recall analysis
# precision = TP / (TP+FP)
# recall = TP / (TP+FN)
confusion_matrix(y_test, (y_score_test_3 > best_threshold_3))

array([[33209, 23374],
       [ 2032,  2885]], dtype=int64)

## 4. Regular Logistic Regression with Top 20

In [70]:
# Top 20 features (actually from the first RF model below)
top20_features = ['EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'DAYS_BIRTH',
 'DAYS_ID_PUBLISH',
 'DAYS_REGISTRATION',
 'AMT_PAYMENT_pay',
 'AMT_INSTALMENT_pay',
 'AMT_ANNUITY',
 'DPD_avg_pay',
 'DAYS_EMPLOYED',
 'DAYS_LAST_PHONE_CHANGE',
 'EXT_SOURCE_1',
 'AMT_CREDIT',
 'DPD_min_pay',
 'AMT_APPLICATION',
 'MIN_DAYS_DECISION',
 'AMT_APPROVED',
 'NUM_INSTALMENT_NUMBER_pay',
 'REGION_POPULATION_RELATIVE',
 'AMT_INCOME_TOTAL']
X_train_select = X_train_nona[top20_features]
X_test_select = X_test_nona[top20_features]

In [71]:
lr4 = LogisticRegression()
lr4.fit(X_train_select, y_train)

C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [72]:
prediction_train_4 = lr4.predict(X_train_select)
prediction_test_4 = lr4.predict(X_test_select)
y_score_test_4 = lr4.predict_proba(X_test_select)[:,1]
print("training accuracy:", metrics.accuracy_score(y_train, prediction_train_4),"\n"
      "test accuracy:", metrics.accuracy_score(y_test, prediction_test_4), "\n"
      "test ROC AUC score:", roc_auc_score(y_test,y_score_test_4), "\n"
     "test F1 score:", f1_score(y_test, prediction_test_4))

training accuracy: 0.919032191187769 
test accuracy: 0.9199674796747968 
test ROC AUC score: 0.6114980316683416 
test F1 score: 0.0


In [73]:
best_threshold_4 = Find_Optimal_Cutoff(y_test, y_score_test_4)
best_f1_score_4 = f1_score(y_test, (y_score_test_4 > best_threshold_4))
print("The best F1 score", round(best_f1_score_4,8), "is achieved with the threshold of", round(best_threshold_4[0], 8))

The best F1 score 0.18320513 is achieved with the threshold of 0.08162441


C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated


In [124]:
# for precision/recall analysis
# precision = TP / (TP+FP)
# recall = TP / (TP+FN)
confusion_matrix(y_test, (y_score_test_4 > best_threshold_4))

array([[33039, 23544],
       [ 2047,  2870]], dtype=int64)

## 5. Logistic Regression with balanced weight with Top 20

In [74]:
lr5 = LogisticRegression(class_weight='balanced')
lr5.fit(X_train_select, y_train)

C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [75]:
prediction_train_5 = lr5.predict(X_train_select)
prediction_test_5 = lr5.predict(X_test_select)
y_score_test_5 = lr5.predict_proba(X_test_select)[:,1]
print("training accuracy:", metrics.accuracy_score(y_train, prediction_train_5),"\n"
      "test accuracy:", metrics.accuracy_score(y_test, prediction_test_5), "\n"
      "test ROC AUC score:", roc_auc_score(y_test,y_score_test_5), "\n"
     "test F1 score:", f1_score(y_test, prediction_test_5))

training accuracy: 0.5957097386574742 
test accuracy: 0.5945528455284553 
test ROC AUC score: 0.5935854305591368 
test F1 score: 0.17627432195830994


In [76]:
best_threshold_5 = Find_Optimal_Cutoff(y_test, y_score_test_5)
best_f1_score_5 = f1_score(y_test, (y_score_test_5 > best_threshold_5))
print("The best F1 score", round(best_f1_score_5,8), "is achieved with the threshold of", round(best_threshold_5[0], 8))

# again, balanced weight doesn't need a readjusted threshold

The best F1 score 0.17653512 is achieved with the threshold of 0.49563569


C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated


In [125]:
# for precision/recall analysis
# precision = TP / (TP+FP)
# recall = TP / (TP+FN)
confusion_matrix(y_test, (y_score_test_5 > best_threshold_5))

array([[32413, 24170],
       [ 2101,  2816]], dtype=int64)

## 6. Logistic Regression with SMOTED, Top 20 dataset

In [77]:
X_train_select_S = X_train_S[top20_features]
X_test_select_S = X_test_S[top20_features]

In [78]:
lr6 = LogisticRegression()
lr6.fit(X_train_select_S, y_train_S)

C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [79]:
prediction_train_6 = lr6.predict(X_train_select_S)
prediction_test_6 = lr6.predict(X_test_select_S)
y_score_test_6 = lr6.predict_proba(X_test_select_S)[:,1]
print("training accuracy:", metrics.accuracy_score(y_train_S, prediction_train_6),"\n"
      "test accuracy:", metrics.accuracy_score(y_test, prediction_test_6), "\n"
      "test ROC AUC score:", roc_auc_score(y_test,y_score_test_6), "\n"
     "test F1 score:", f1_score(y_test, prediction_test_6))

training accuracy: 0.5831722624960746 
test accuracy: 0.5866666666666667 
test ROC AUC score: 0.5928215743985581 
test F1 score: 0.17569232764770737


In [80]:
best_threshold_6 = Find_Optimal_Cutoff(y_test, y_score_test_6)
best_f1_score_6 = f1_score(y_test, (y_score_test_6 > best_threshold_6))
print("The best F1 score", round(best_f1_score_6,8), "is achieved with the threshold of", round(best_threshold_6[0], 8))

# again, no need to adjust threshold for a SMOTED dataset

The best F1 score 0.17614828 is achieved with the threshold of 0.49658826


C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated


In [126]:
# for precision/recall analysis
# precision = TP / (TP+FP)
# recall = TP / (TP+FN)
confusion_matrix(y_test, (y_score_test_6 > best_threshold_6))

array([[32374, 24209],
       [ 2104,  2813]], dtype=int64)

## 7. Logistic with top 20 features SMOTED (SMOTED after choosing)

In [104]:
# Michael's dataset
X_train_select_S2 = pd.read_csv('cleaned_tables/x20_train.csv', index_col = 0)
X_test_select_S2 = pd.read_csv('cleaned_tables/x20_test.csv', index_col = 0)
y_train_select_S2 = pd.read_csv('cleaned_tables/y20_train.csv', index_col = 0, header = None)
y_test_select_S2 = pd.read_csv('cleaned_tables/y20_test.csv', index_col = 0, header = None)

In [105]:
lr7 = LogisticRegression()
lr7.fit(X_train_select_S2, y_train_select_S2)

C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [106]:
prediction_train_7 = lr7.predict(X_train_select_S2)
prediction_test_7 = lr7.predict(X_test_select_S2)
y_score_test_7 = lr7.predict_proba(X_test_select_S2)[:,1]
print("training accuracy:", metrics.accuracy_score(y_train_select_S2, prediction_train_7),"\n"
      "test accuracy:", metrics.accuracy_score(y_test_select_S2, prediction_test_7), "\n"
      "test ROC AUC score:", roc_auc_score(y_test_select_S2,y_score_test_7), "\n"
     "test F1 score:", f1_score(y_test_select_S2, prediction_test_7))

training accuracy: 0.5798859751162143 
test accuracy: 0.5882601626016261 
test ROC AUC score: 0.5924654946969022 
test F1 score: 0.17603800598724456


In [107]:
best_threshold_7 = Find_Optimal_Cutoff(y_test_select_S2, y_score_test_7)
best_f1_score_7 = f1_score(y_test, (y_score_test_7 > best_threshold_7))
print("The best F1 score", round(best_f1_score_7,8), "is achieved with the threshold of", round(best_threshold_7[0], 8))

# again, no need to adjust threshold for a SMOTED dataset

The best F1 score 0.176001 is achieved with the threshold of 0.49622531


C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated


In [127]:
# for precision/recall analysis
# precision = TP / (TP+FP)
# recall = TP / (TP+FN)
confusion_matrix(y_test_select_S2, (y_score_test_7 > best_threshold_7))

array([[32368, 24215],
       [ 2106,  2811]], dtype=int64)

# Random Forest

## 1. Regular Random Forest

In [81]:
from sklearn.ensemble import RandomForestClassifier
RF_1 = RandomForestClassifier(random_state=0, n_estimators = 200) 
RF_1.fit(X_train_nona,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [82]:
from sklearn.metrics import roc_auc_score, f1_score
from sklearn import metrics

y_pred_1 = RF_1.predict(X_train_nona)
y_pred_test_1 = RF_1.predict(X_test_nona)
y_test_score_1 = RF_1.predict_proba(X_test_nona)[:,1]
print("training accuracy:", round(metrics.accuracy_score(y_train,y_pred_1),10), "\n"
     "test accuracy:", round(metrics.accuracy_score(y_test,y_pred_test_1),10), "\n"
    "ROC_AUC_score:", round(roc_auc_score(y_test,y_test_score_1),10), "\n"
     "F1 score:", round(f1_score(y_test, y_pred_test_1),10))

training accuracy: 1.0 
test accuracy: 0.920097561 
ROC_AUC_score: 0.7377949763 
F1 score: 0.0012195122


In [83]:
best_threshold_RF_1 = Find_Optimal_Cutoff(y_test, y_test_score_1)
best_f1_score_RF_1 = f1_score(y_test, (y_test_score_1 > best_threshold_RF_1))
print("The best F1 score", round(best_f1_score_RF_1,8), "is achieved with the threshold of", round(best_threshold_RF_1[0], 8))

The best F1 score 0.25871775 is achieved with the threshold of 0.1


C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated


In [128]:
# for precision/recall analysis
# precision = TP / (TP+FP)
# recall = TP / (TP+FN)
confusion_matrix(y_test, (y_test_score_1 >= best_threshold_RF_1))

array([[38717, 17866],
       [ 1620,  3297]], dtype=int64)

In [84]:
feature_importances = pd.DataFrame(RF_1.feature_importances_, index = X_train_nona.columns, columns=['importance']).sort_values('importance', ascending=False)
feature_importances[:50].index

Index(['EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'DAYS_ID_PUBLISH',
       'DAYS_REGISTRATION', 'AMT_INSTALMENT_pay', 'AMT_PAYMENT_pay',
       'DPD_avg_pay', 'AMT_ANNUITY', 'DAYS_EMPLOYED', 'DAYS_LAST_PHONE_CHANGE',
       'EXT_SOURCE_1', 'AMT_CREDIT', 'DPD_min_pay', 'AMT_APPLICATION',
       'NUM_INSTALMENT_NUMBER_pay', 'MIN_DAYS_DECISION', 'AMT_APPROVED',
       'REGION_POPULATION_RELATIVE', 'AMT_INCOME_TOTAL', 'AMT_GOODS_PRICE',
       'DPD_max_pay', 'mean_term', 'HOUR_APPR_PROCESS_START',
       'MAX_BU_DAYS_CREDIT_ENDDATE', 'PERCENT_OWED_pay', 'AVG_BU_DAYS_CREDIT',
       'no_DPD_Percentage_pay', 'DAYS_TERMINATION', 'SUM_BU_AMT_CREDIT_SUM',
       'OWED_pay', 'NUM_APPLICATIONS', 'PCT_APPROVED',
       'PERCENT_UNDER_PAID_pay', 'TOTALAREA_MODE', 'LIVINGAREA_MEDI',
       'max_term', 'OBS_30_CNT_SOCIAL_CIRCLE', 'APARTMENTS_MEDI',
       'COUNT_BU_CREDIT_ACTIVE', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'DPD_Count_pay',
       'min_term', 'OWN_CAR_AGE', 'LANDAREA_MEDI', 'Percent_BU_CREDIT_

## 2. Random Forest with Balanced Weight

In [85]:
RF_2 = RandomForestClassifier(random_state=0, n_estimators = 200, class_weight='balanced') 
RF_2.fit(X_train_nona,y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=200, n_jobs=None, oob_score=False,
                       random_state=0, verbose=0, warm_start=False)

In [86]:
y_pred_2 = RF_2.predict(X_train_nona)
y_pred_test_2 = RF_2.predict(X_test_nona)
y_test_score_2 = RF_2.predict_proba(X_test_nona)[:,1]
print("training accuracy:", round(metrics.accuracy_score(y_train,y_pred_2),10), "\n"
     "test accuracy:", round(metrics.accuracy_score(y_test,y_pred_test_2),10), "\n"
    "ROC_AUC_score:", round(roc_auc_score(y_test,y_test_score_2),10), "\n"
     "F1 score:", round(f1_score(y_test, y_pred_test_2),10))

training accuracy: 1.0 
test accuracy: 0.9199837398 
ROC_AUC_score: 0.744374741 
F1 score: 0.0008121827


In [87]:
best_threshold_RF_2 = Find_Optimal_Cutoff(y_test, y_test_score_2)
best_f1_score_RF_2 = f1_score(y_test, (y_test_score_2 > best_threshold_RF_2))
print("The best F1 score", round(best_f1_score_RF_2,8), "is achieved with the threshold of", round(best_threshold_RF_2[0], 8))

# surprisingly even with balanced weight, RF needs to be adjusted...

The best F1 score 0.2672392 is achieved with the threshold of 0.09


C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated


In [129]:
# for precision/recall analysis
# precision = TP / (TP+FP)
# recall = TP / (TP+FN)
confusion_matrix(y_test, (y_test_score_2 >= best_threshold_RF_2))

array([[39346, 17237],
       [ 1582,  3335]], dtype=int64)

In [88]:
feature_importances = pd.DataFrame(RF_2.feature_importances_, index = X_train_nona.columns, columns=['importance']).sort_values('importance', ascending=False)
feature_importances[:50].index
# top 20 are the same as Michael's

Index(['EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'EXT_SOURCE_1', 'DAYS_LAST_PHONE_CHANGE', 'DAYS_ID_PUBLISH',
       'DAYS_REGISTRATION', 'AMT_ANNUITY', 'AMT_CREDIT', 'AMT_PAYMENT_pay',
       'DPD_avg_pay', 'AMT_GOODS_PRICE', 'AMT_INSTALMENT_pay',
       'REGION_POPULATION_RELATIVE', 'MIN_DAYS_DECISION',
       'NUM_INSTALMENT_NUMBER_pay', 'DPD_min_pay', 'AMT_APPROVED',
       'AMT_APPLICATION', 'AMT_INCOME_TOTAL', 'DPD_max_pay',
       'AVG_BU_DAYS_CREDIT', 'mean_term', 'MAX_BU_DAYS_CREDIT_ENDDATE',
       'HOUR_APPR_PROCESS_START', 'PERCENT_OWED_pay', 'no_DPD_Percentage_pay',
       'SUM_BU_AMT_CREDIT_SUM', 'PCT_APPROVED', 'DAYS_TERMINATION', 'OWED_pay',
       'PERCENT_UNDER_PAID_pay', 'TOTALAREA_MODE', 'LIVINGAREA_MEDI',
       'NUM_APPLICATIONS', 'APARTMENTS_MEDI', 'max_term', 'LANDAREA_MEDI',
       'COUNT_BU_CREDIT_ACTIVE', 'BASEMENTAREA_MEDI', 'OWN_CAR_AGE',
       'Percent_BU_CREDIT_ACTIVE', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DPD_Count_pay',
       'AMT_

In [89]:
# feature_importances[:25].plot(kind='bar')

In [90]:
# %pylab inline
# plt.figure(figsize = (8,8))

# from sklearn.metrics import roc_curve
# fpr, tpr, threshold = roc_curve(y_test, y_score_test)
# plt.plot(fpr, tpr, marker = 'o', markersize = 4)#, linestyle = 'None')
# plt.plot([0, 1], [0, 1],'r--')

## 3. Random Forest with SMOTENC dataset

In [91]:
RF_3 = RandomForestClassifier(random_state=0, n_estimators = 200) 
RF_3.fit(X_train_S,y_train_S)

C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [92]:
y_pred_3 = RF_3.predict(X_train_S)
y_pred_test_3 = RF_3.predict(X_test_S)
y_test_score_3 = RF_3.predict_proba(X_test_S)[:,1]
print("training accuracy:", round(metrics.accuracy_score(y_train_S,y_pred_3),10), "\n"
     "test accuracy:", round(metrics.accuracy_score(y_test,y_pred_test_3),10), "\n"
    "ROC_AUC_score:", round(roc_auc_score(y_test,y_test_score_3),10), "\n"
     "F1 score:", round(f1_score(y_test, y_pred_test_3),10))

training accuracy: 1.0 
test accuracy: 0.9199186992 
ROC_AUC_score: 0.7139714604 
F1 score: 0.0056531395


In [93]:
best_threshold_RF_3 = Find_Optimal_Cutoff(y_test, y_test_score_3)
best_f1_score_RF_3 = f1_score(y_test, (y_test_score_3 > best_threshold_RF_3))
print("The best F1 score", round(best_f1_score_RF_3,8), "is achieved with the threshold of", round(best_threshold_RF_3[0], 8))

The best F1 score 0.23883429 is achieved with the threshold of 0.18


C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated


In [130]:
# for precision/recall analysis
# precision = TP / (TP+FP)
# recall = TP / (TP+FN)
confusion_matrix(y_test, (y_test_score_3 >= best_threshold_RF_3))

array([[37272, 19311],
       [ 1684,  3233]], dtype=int64)

In [94]:
feature_importances = pd.DataFrame(RF_3.feature_importances_, index = X_train_S.columns, columns=['importance']).sort_values('importance', ascending=False)
feature_importances[:50].index

# these are different from the above; but the metrics are also not as good

Index(['EXT_SOURCE_2', 'EXT_SOURCE_3', 'CNT_FAM_MEMBERS', 'CNT_CHILDREN',
       'OBS_30_CNT_SOCIAL_CIRCLE', 'NAME_EDUCATION_TYPE_Higher education',
       'DPD_Count_pay', 'WALLSMATERIAL_MODE_Panel', 'EXT_SOURCE_1',
       'no_DPD_Percentage_pay', 'DPD_max_pay',
       'WEEKDAY_APPR_PROCESS_START_MONDAY',
       'NAME_EDUCATION_TYPE_Secondary / secondary special',
       'PERCENT_UNDER_PAID_pay', 'WEEKDAY_APPR_PROCESS_START_THURSDAY',
       'WALLSMATERIAL_MODE_Stone, brick',
       'WEEKDAY_APPR_PROCESS_START_WEDNESDAY',
       'WEEKDAY_APPR_PROCESS_START_TUESDAY',
       'WEEKDAY_APPR_PROCESS_START_FRIDAY',
       'WEEKDAY_APPR_PROCESS_START_SATURDAY', 'DAYS_LAST_PHONE_CHANGE',
       'DAYS_ID_PUBLISH', 'FLOORSMAX_MEDI', 'DAYS_BIRTH', 'TOTALAREA_MODE',
       'DEF_30_CNT_SOCIAL_CIRCLE', 'FLAG_PHONE', 'REGION_POPULATION_RELATIVE',
       'OCCUPATION_TYPE_Unemployed', 'PCT_APPROVED', 'AMT_PAYMENT_pay',
       'PERCENT_OWED_pay', 'NAME_TYPE_SUITE_Unaccompanied',
       'AMT_INSTALMENT_

## 4. Regular Random Forest with Top 20 features

In [95]:
RF_4 = RandomForestClassifier(random_state=0, n_estimators = 200)
RF_4.fit(X_train_select, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [96]:
y_pred_4 = RF_4.predict(X_train_select)
y_pred_test_4 = RF_4.predict(X_test_select)
y_test_score_4 = RF_4.predict_proba(X_test_select)[:,1]
print("training accuracy:", round(metrics.accuracy_score(y_train,y_pred_4),10), "\n"
     "test accuracy:", round(metrics.accuracy_score(y_test,y_pred_test_4),10), "\n"
    "ROC_AUC_score:", round(roc_auc_score(y_test,y_test_score_4),10), "\n"
     "F1 score:", round(f1_score(y_test, y_pred_test_4),10))

best_threshold_RF_4 = Find_Optimal_Cutoff(y_test, y_test_score_4)
best_f1_score_RF_4 = f1_score(y_test, (y_test_score_4 > best_threshold_RF_4))
print("The best F1 score", round(best_f1_score_RF_4,8), "is achieved with the threshold of", round(best_threshold_RF_4[0], 8))

training accuracy: 1.0 
test accuracy: 0.9201138211 
ROC_AUC_score: 0.7257887917 
F1 score: 0.0096754687
The best F1 score 0.24701926 is achieved with the threshold of 0.095


C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated


In [131]:
# for precision/recall analysis
# precision = TP / (TP+FP)
# recall = TP / (TP+FN)
confusion_matrix(y_test, (y_test_score_4 >= best_threshold_RF_4))

array([[37295, 19288],
       [ 1570,  3347]], dtype=int64)

## 5. Random Forest with Balanced Weight with Top 20 features

In [97]:
RF_5 = RandomForestClassifier(random_state=0, n_estimators = 200, class_weight='balanced') 
RF_5.fit(X_train_select, y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=200, n_jobs=None, oob_score=False,
                       random_state=0, verbose=0, warm_start=False)

In [98]:
y_pred_5 = RF_5.predict(X_train_select)
y_pred_test_5 = RF_5.predict(X_test_select)
y_test_score_5 = RF_5.predict_proba(X_test_select)[:,1]
print("training accuracy:", round(metrics.accuracy_score(y_train,y_pred_5),10), "\n"
     "test accuracy:", round(metrics.accuracy_score(y_test,y_pred_test_5),10), "\n"
    "ROC_AUC_score:", round(roc_auc_score(y_test,y_test_score_5),10), "\n"
     "F1 score:", round(f1_score(y_test, y_pred_test_5),10))

best_threshold_RF_5 = Find_Optimal_Cutoff(y_test, y_test_score_5)
best_f1_score_RF_5 = f1_score(y_test, (y_test_score_5 > best_threshold_RF_5))
print("The best F1 score", round(best_f1_score_RF_5,8), "is achieved with the threshold of", round(best_threshold_RF_5[0], 8))

training accuracy: 1.0 
test accuracy: 0.9199512195 
ROC_AUC_score: 0.7342181918 
F1 score: 0.0060569352
The best F1 score 0.25759257 is achieved with the threshold of 0.085


C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated


In [132]:
# for precision/recall analysis
# precision = TP / (TP+FP)
# recall = TP / (TP+FN)
confusion_matrix(y_test, (y_test_score_5 >= best_threshold_RF_5))

array([[38897, 17686],
       [ 1661,  3256]], dtype=int64)

## 6. Random Forest with SMOTED dataset with Top 20 features

In [99]:
RF_6 = RandomForestClassifier(random_state=0, n_estimators = 200) 
RF_6.fit(X_train_select_S, y_train_S)

C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [100]:
y_pred_6 = RF_6.predict(X_train_select_S)
y_pred_test_6 = RF_6.predict(X_test_select_S)
y_test_score_6 = RF_6.predict_proba(X_test_select_S)[:,1]
print("training accuracy:", round(metrics.accuracy_score(y_train_S,y_pred_6),10), "\n"
     "test accuracy:", round(metrics.accuracy_score(y_test,y_pred_test_6),10), "\n"
    "ROC_AUC_score:", round(roc_auc_score(y_test,y_test_score_6),10), "\n"
     "F1 score:", round(f1_score(y_test, y_pred_test_6),10))

best_threshold_RF_6 = Find_Optimal_Cutoff(y_test, y_test_score_6)
best_f1_score_RF_6 = f1_score(y_test, (y_test_score_6 > best_threshold_RF_6))
print("The best F1 score", round(best_f1_score_RF_6,8), "is achieved with the threshold of", round(best_threshold_RF_6[0], 8))

training accuracy: 1.0 
test accuracy: 0.8537886179 
ROC_AUC_score: 0.6876861933 
F1 score: 0.1877145438
The best F1 score 0.22158948 is achieved with the threshold of 0.27


C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated


In [133]:
# for precision/recall analysis
# precision = TP / (TP+FP)
# recall = TP / (TP+FN)
confusion_matrix(y_test, (y_test_score_6 >= best_threshold_RF_6))

array([[36309, 20274],
       [ 1789,  3128]], dtype=int64)

## 7. Using the Top 20 features to SMOTENC, then train another RF

**with Michael's dataset**

In [ ]:
# # SMOTENC Top 20

# X_train_select_copy = X_train_select.copy()

# #Turning all categorical columns into object type
# for column in X_train_select_copy.columns:
#     if(len(X_train_select_copy[column].value_counts())<=2):
#         X_train_select_copy[column] = X_train_select_copy[column].astype('object')
        
# #return column index of categorical columns
# cat_col_index = []
# for column in X_train_select_copy.columns:
#     if(X_train_select_copy[column].dtype == 'object'):
#         cat_col_index.append(X_train_select_copy.columns.get_loc(column))
        
# from imblearn.over_sampling import SMOTENC
# smt = SMOTENC(categorical_features=cat_col_index,random_state = 11)
# X_train_select_2S, y_train = smt.fit_sample(X_train_select_copy, y_train)

In [ ]:
prediction_train_7 = lr7.predict(X_train_select_S2)
prediction_test_7 = lr7.predict(X_test_select_S2)
y_score_test_7 = lr7.predict_proba(X_test_select_S2)[:,1]
print("training accuracy:", metrics.accuracy_score(y_train_select_S2, prediction_train_7),"\n"
      "test accuracy:", metrics.accuracy_score(y_test_select_S2, prediction_test_7), "\n"
      "test ROC AUC score:", roc_auc_score(y_test_select_S2,y_score_test_7), "\n"
     "test F1 score:", f1_score(y_test_select_S2, prediction_test_7))

best_threshold_7 = Find_Optimal_Cutoff(y_test_select_S2, y_score_test_7)
best_f1_score_7 = f1_score(y_test, (y_score_test_7 > best_threshold_7))
print("The best F1 score", round(best_f1_score_7,8), "is achieved with the threshold of", round(best_threshold_7[0], 8))

# again, no need to adjust threshold for a SMOTED dataset

In [108]:
RF_7 = RandomForestClassifier(random_state=0, n_estimators = 200) 
RF_7.fit(X_train_select_S2, y_train_select_S2)

C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [112]:
y_pred_7 = RF_7.predict(X_train_select_S2)
y_pred_test_7 = RF_7.predict(X_test_select_S2)
y_test_score_7 = RF_7.predict_proba(X_test_select_S2)[:,1]
print("training accuracy:", round(metrics.accuracy_score(y_train_select_S2,y_pred_7),10), "\n"
     "test accuracy:", round(metrics.accuracy_score(y_test_select_S2,y_pred_test_7),10), "\n"
    "ROC_AUC_score:", round(roc_auc_score(y_test_select_S2,y_test_score_7),10), "\n"
     "F1 score:", round(f1_score(y_test_select_S2, y_pred_test_7),10))

best_threshold_RF_7 = Find_Optimal_Cutoff(y_test_select_S2, y_test_score_7)
best_f1_score_RF_7 = f1_score(y_test_select_S2, (y_test_score_7 > best_threshold_RF_7))
print("The best F1 score", round(best_f1_score_RF_7,8), "is achieved with the threshold of", round(best_threshold_RF_7[0], 8))

training accuracy: 1.0 
test accuracy: 0.8523252033 
ROC_AUC_score: 0.6827020138 
F1 score: 0.1888174348
The best F1 score 0.21882623 is achieved with the threshold of 0.265


C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated


In [119]:
# for precision/recall analysis
# precision = TP / (TP+FP)
# recall = TP / (TP+FN)
confusion_matrix(y_test_select_S2, (y_test_score_7 >= best_threshold_RF_7))

array([[36148, 20435],
       [ 1823,  3094]], dtype=int64)

# GLRM - Need to be done in AIX360

#### Rerun all before Modeling; then come here

## 1. Train Logistic Rule with the top 20 features in the regular dataset

In [134]:
# top 20 features from the first random forest model
top20_features = ['EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'DAYS_BIRTH',
 'DAYS_ID_PUBLISH',
 'DAYS_REGISTRATION',
 'AMT_PAYMENT_pay',
 'AMT_INSTALMENT_pay',
 'AMT_ANNUITY',
 'DPD_avg_pay',
 'DAYS_EMPLOYED',
 'DAYS_LAST_PHONE_CHANGE',
 'EXT_SOURCE_1',
 'AMT_CREDIT',
 'DPD_min_pay',
 'AMT_APPLICATION',
 'MIN_DAYS_DECISION',
 'AMT_APPROVED',
 'NUM_INSTALMENT_NUMBER_pay',
 'REGION_POPULATION_RELATIVE',
 'AMT_INCOME_TOTAL']
X_train_select = X_train_nona[top20_features]
X_test_select = X_test_nona[top20_features]

In [135]:
print(X_train_select.shape, X_test_select.shape)

(245999, 20) (61500, 20)


### Binarize data

In [136]:
# https://github.com/IBM/AIX360/blob/master/examples/rbm/breast-cancer-logistic.ipynb
# https://github.com/IBM/AIX360/blob/master/aix360/algorithms/rbm/features.py
from aix360.algorithms.rbm import FeatureBinarizer

fb = FeatureBinarizer(negations=True) 
X_train_fb = fb.fit_transform(X_train_select)
X_test_fb = fb.transform(X_test_select)

Skipping column 'DAYS_BIRTH': data type cannot be handled
Skipping column 'DAYS_ID_PUBLISH': data type cannot be handled
Skipping column 'DAYS_EMPLOYED': data type cannot be handled
Skipping column 'DAYS_BIRTH': data type cannot be handled
Skipping column 'DAYS_ID_PUBLISH': data type cannot be handled
Skipping column 'DAYS_EMPLOYED': data type cannot be handled
Skipping column 'DAYS_BIRTH': data type cannot be handled
Skipping column 'DAYS_ID_PUBLISH': data type cannot be handled
Skipping column 'DAYS_EMPLOYED': data type cannot be handled


It seems like when columns are skipped, they just don't make it into the training table at all:

In [138]:
X_train_select.dtypes # problematic columns are int64

EXT_SOURCE_2                  float64
EXT_SOURCE_3                  float64
DAYS_BIRTH                      int64
DAYS_ID_PUBLISH                 int64
DAYS_REGISTRATION             float64
AMT_PAYMENT_pay               float64
AMT_INSTALMENT_pay            float64
AMT_ANNUITY                   float64
DPD_avg_pay                   float64
DAYS_EMPLOYED                   int64
DAYS_LAST_PHONE_CHANGE        float64
EXT_SOURCE_1                  float64
AMT_CREDIT                    float64
DPD_min_pay                   float64
AMT_APPLICATION               float64
MIN_DAYS_DECISION             float64
AMT_APPROVED                  float64
NUM_INSTALMENT_NUMBER_pay     float64
REGION_POPULATION_RELATIVE    float64
AMT_INCOME_TOTAL              float64
dtype: object

In [140]:
X_train_select_GLRM = X_train_nona[top20_features].astype(float)
X_test_select_GLRM = X_test_nona[top20_features].astype(float)
X_train_select_GLRM.dtypes

EXT_SOURCE_2                  float64
EXT_SOURCE_3                  float64
DAYS_BIRTH                    float64
DAYS_ID_PUBLISH               float64
DAYS_REGISTRATION             float64
AMT_PAYMENT_pay               float64
AMT_INSTALMENT_pay            float64
AMT_ANNUITY                   float64
DPD_avg_pay                   float64
DAYS_EMPLOYED                 float64
DAYS_LAST_PHONE_CHANGE        float64
EXT_SOURCE_1                  float64
AMT_CREDIT                    float64
DPD_min_pay                   float64
AMT_APPLICATION               float64
MIN_DAYS_DECISION             float64
AMT_APPROVED                  float64
NUM_INSTALMENT_NUMBER_pay     float64
REGION_POPULATION_RELATIVE    float64
AMT_INCOME_TOTAL              float64
dtype: object

In [148]:
fb = FeatureBinarizer(negations=True) 
X_train_fb = fb.fit_transform(X_train_select_GLRM)
X_test_fb = fb.transform(X_test_select_GLRM)

In [149]:
X_train_fb['DAYS_BIRTH'][:8]

operation       <=                                                        \
value     -22184.0 -20482.0 -18904.0 -17235.0 -15757.0 -14426.0 -13152.0   
45907            0        0        0        1        1        1        1   
104934           0        0        0        0        0        0        1   
256686           0        0        1        1        1        1        1   
287721           0        0        0        0        0        0        0   
238654           0        0        0        0        1        1        1   
166047           0        0        0        1        1        1        1   
168942           0        0        0        0        0        0        1   
63275            0        0        0        0        0        0        0   

operation                          >                                      \
value     -11705.0 -10290.0 -22184.0 -20482.0 -18904.0 -17235.0 -15757.0   
45907            1        1        1        1        1        0        0   
104934           1        1        1        1        1        1        1   
256686           1        1        1        1        0        0        0   
287721           0        1        1        1        1        1        1   
238654           1        1        1        1        1        1        0   
166047           1        1        1        1        1        0        0   
168942           1        1        1        1        1        1        1   
63275            0        1        1        1        1        1        1   

operation                                      
value     -14426.0 -13152.0 -11705.0 -10290.0  
45907            0        0        0        0  
104934           1        0        0        0  
256686           0        0        0        0  
287721           1        1        1        0  
238654           0        0        0        0  
166047           0        0        0        0  
168942           1        0        0        0  
63275            1        1        1        0

### Train LogisticRuleRegression

In [150]:
from aix360.algorithms.rbm import GLRMExplainer, LogisticRuleRegression
logistic_model = LogisticRuleRegression(maxSolverIter = 2000)
explainer = GLRMExplainer(logistic_model)
explainer.fit(X_train_fb, y_train)

In [152]:
Y_pred = explainer.predict(X_test_fb)
Y_score = explainer.predict_proba(X_test_fb)

from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
print("AUROC score:", roc_auc_score(y_test, Y_score), "\n"
      "F1 score:", f1_score(y_test, Y_pred), "\n"
      "Precision:", precision_score(y_test, Y_pred), "\n"
      "Recall:", recall_score(y_test, Y_pred)
     )

AUROC score: 0.5 
F1 score: 0.0 
Precision: 0.0 
Recall: 0.0


C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Jenny\Anaconda3\envs\aix360\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [144]:
explainer.explain()

,rule,coefficient
0,(intercept),-2.42929


## It does not seem that the Logistic Rule classifier used any info in the un-SMOTED dataset...

## 2. Train Logistic Rule with the top 20 features in Michael's SMOTED dataset

In [153]:
from aix360.algorithms.rbm import FeatureBinarizer

fb = FeatureBinarizer(negations=True) 
X_train_fb_2 = fb.fit_transform(X_train_select_S2)
X_test_fb_2 = fb.transform(X_test_select_S2)

In [155]:
X_train_select_S2.dtypes

EXT_SOURCE_2                  float64
EXT_SOURCE_3                  float64
DAYS_BIRTH                    float64
DAYS_ID_PUBLISH               float64
DAYS_REGISTRATION             float64
AMT_PAYMENT_pay               float64
AMT_INSTALMENT_pay            float64
AMT_ANNUITY                   float64
DPD_avg_pay                   float64
DAYS_EMPLOYED                 float64
DAYS_LAST_PHONE_CHANGE        float64
EXT_SOURCE_1                  float64
AMT_CREDIT                    float64
DPD_min_pay                   float64
AMT_APPLICATION               float64
MIN_DAYS_DECISION             float64
AMT_APPROVED                  float64
NUM_INSTALMENT_NUMBER_pay     float64
REGION_POPULATION_RELATIVE    float64
AMT_INCOME_TOTAL              float64
dtype: object

In [156]:
X_train_fb_2['DAYS_BIRTH'][:8]

operation            <=                                            \
value     -21662.963781 -19594.000000 -17748.000000 -16232.195779   
0                     0             0             1             1   
1                     0             0             0             0   
2                     0             0             1             1   
3                     0             0             0             0   
4                     0             0             0             1   
5                     0             0             0             1   
6                     0             0             0             0   
7                     0             0             0             0   

operation                                                          \
value     -14971.231383 -13828.696526 -12714.612311 -11557.000000   
0                     1             1             1             1   
1                     0             0             1             1   
2                     1             1             1             1   
3                     0             0             0             0   
4                     1             1             1             1   
5                     1             1             1             1   
6                     0             0             1             1   
7                     0             0             0             0   

operation                           >                              \
value     -10322.000000 -21662.963781 -19594.000000 -17748.000000   
0                     1             1             1             0   
1                     1             1             1             1   
2                     1             1             1             0   
3                     1             1             1             1   
4                     1             1             1             1   
5                     1             1             1             1   
6                     1             1             1             1   
7                     1             1             1             1   

operation                                                          \
value     -16232.195779 -14971.231383 -13828.696526 -12714.612311   
0                     0             0             0             0   
1                     1             1             1             0   
2                     0             0             0             0   
3                     1             1             1             1   
4                     0             0             0             0   
5                     0             0             0             0   
6                     1             1             1             0   
7                     1             1             1             1   

operation                              
value     -11557.000000 -10322.000000  
0                     0             0  
1                     0             0  
2                     0             0  
3                     1             0  
4                     0             0  
5                     0             0  
6                     0             0  
7                     1             0

### Train LogisticRuleRegression

In [160]:
from aix360.algorithms.rbm import GLRMExplainer, LogisticRuleRegression
logistic_model = LogisticRuleRegression(maxSolverIter = 2000)
explainer = GLRMExplainer(logistic_model)
explainer.fit(X_train_fb_2, y_train_select_S2.loc[:,1]) # seems like this model needs y_train needs to be a series

In [162]:
Y_pred_2 = explainer.predict(X_test_fb_2)
Y_score_2 = explainer.predict_proba(X_test_fb_2)

from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
print("AUROC score:", roc_auc_score(y_test_select_S2, Y_score_2), "\n"
      "F1 score:", f1_score(y_test_select_S2, Y_pred_2), "\n"
      "Precision:", precision_score(y_test_select_S2, Y_pred_2), "\n"
      "Recall:", recall_score(y_test_select_S2, Y_pred_2)
     )

AUROC score: 0.6678057601258027 
F1 score: 0.21295155020384943 
Precision: 0.12606645711719802 
Recall: 0.6851738865161684


In [164]:
explainer.explain()

,rule,coefficient
0,(intercept),-1.58187
1,EXT_SOURCE_2 <= 0.65 AND EXT_SOURCE_3 <= 0.61,1.181
2,EXT_SOURCE_3 <= 0.51,1.11591
3,EXT_SOURCE_2 <= 0.55,1.0389
4,EXT_SOURCE_2 <= 0.65 AND EXT_SOURCE_3 <= 0.51,0.0790746


In [165]:
# for precision/recall analysis
# precision = TP / (TP+FP)
# recall = TP / (TP+FN)
confusion_matrix(y_test_select_S2, Y_pred_2)

array([[33228, 23355],
       [ 1548,  3369]], dtype=int64)